## Modular arithmetic essentials
### Benchmarks

In [1]:
import time

s = time.time()
ans = 1002583 ** 939001 % 2003951
t = time.time()

print("calculating {} took {:.2f} seconds".format(ans,t - s))

calculating 1590868 took 7.31 seconds


In [2]:
s = time.time()
ans = pow(1002583,939001,2003951)
t = time.time()

print("calculating {} operation took {:.2f} seconds".format( ans, t-s))

calculating 1590868 operation took 0.00 seconds


### Encrypting a number

If we start with a generator `g` and raise it to a power `s` mod `n`, then in general we cannot determine `s` even if `g` and `n` are known. This is the discrete logarithm problem.

We can say we "encrypt" `s` using the scheme

$$ E(s) = g^{s} \pmod n $$

But this means we need to choose `g` and `n` carefully. If `g` is 3, `s` is 2, and `n` is a large number, we can intelligently guess what `s` is in this case. We want to pick `g` and `n` such that all members "wrap around." We won't concern ourselves with this here, but you should be aware of it. The other issue is not picking a sufficiently large `n`. 

### Computing the discrete logarithm
If we do `s_p = g^s mod n`, and make `g` and `n` public, then it is infeasible to compute `s`. But this is only true if `n` is sufficiently large.

In the following example, we provide `s_p`, `g`, and `n`. Use python to bruteforce the solution to find `s`.

In [3]:
n = 9551
g = 5
encrypted_number = 5666

How to use python to bruteforce the solution to find `s`

In [5]:

for i in range(1,n):
    if pow(g,i,n) == encrypted_number:
        solution = i
        break
print("solution is {}".format(solution))

solution is 2531


In [6]:
assert pow(g, solution, n) == encrypted_number
print("solution is {}".format(solution))

solution is 2531


### Computing the discrete logarithm the smart way
The smart way to do optimal algorithms is to [copy and paste from stackoverflow](https://stackoverflow.com/a/58665206/2079806). There is no need to understand how the library works

<font color='red'>Break the cryptography below and obtain the secret number</font>

In [7]:
n = 1000004119
g = 5
encrypted_number = 767805982

In [8]:

from sympy.ntheory import discrete_log

for i in range(1,n):
    if discrete_log(n,encrypted_number,g) == i:
        solution = i
        break
print("solution is {}".format(solution))

solution is 420


In [9]:
assert pow(g, solution, n) == encrypted_number
print("solution is {}".format(solution))

solution is 420


## Zero Knowledge Addition
The following property is very important. It lets us verify the addition of numbers without knowing the numbers.

$$ g^{a}g^{b} = g^{a + b} \pmod n $$

Try a few different values of `a` and `b` yourself to see it in action

In [11]:
n = 1000004119
g = 5
a = 5 # put your own value
b = 555 # put your own value
c = a + b

print(pow(g, a, n) * pow(g, b, n) == pow(g, a + b, n))

False


Wait what?! That's supposed to be true?


<font color='red'>The code above has a bug. What is the bug?</font>

In [12]:
# The multiplication operation (*) happens before the modular reduction (% n)
# Your corrected code here
n = 1000004119
g = 5
a = 5  # Put your own value
b = 555  # Put your own value
c = a + b

result = (pow(g, a, n) * pow(g, b, n)) % n

print(result == pow(g, c, n))


True


## Zero Knowledge Subtraction
We can also encrypt the operation a - b. This is the same as 

$$ g^{a}g^{-b} = g^{a-b} $$

But we can't just stick a negative sign in front of the exponent

In [13]:
g = 57
n = 101
g ** -5 % n

1.6619797259514097e-09

The above is not an integer! However, python is smart enough to know what you are doing if you use `pow`. To test this, we expect

$$ g^{-a}g^{a} = 1 $$

because g^0 is 1, and `a - a = 0`.

In [14]:
a = 22
pow(g, a, n) * pow(g, -a, n) % n == 1

True

So what magic is happening behind the scenes? The formula below is used to compute the modular inverse.

$$ a^{-1} = a^{n - 2} \pmod n $$

In [15]:
a_inv = a ** (n - 2) % n
a_inv * a % n == 1

True

## Multiplication by a constant
Multiplication by a constant is really just (modern zk proof algorithms do not use this technique, this is just for illustration purposes. 4x is really x + x + x + x. In encrypted form, this would be

$$ g^{4a} = g^{a} g^{a} g^{a} g^{a} $$

This allows us to "multiply by a constant" in a zero knowledge fashion.

In [16]:
a = 15
pow(g, a, n) * pow(g, a, n) * pow(g, a, n) * pow(g, a, n) % n == pow(g, a * 4, n)

True

Of course, it would be annoying to multiply like that if you have a big coefficient, so the following is better

In [17]:
pow(pow(g, a, n), 4 , n) == pow(g, a * 4, n)

True

## I know the solution to a systems of equations

Now you should be able to prove to me that you know the solution to in a zero knowledge fashion

$$
2x + 8y = 7944\\
5x + 3y = 4764
$$

<font color='red'>**Create a zk proof system where you can prove to a verifier (with an agreed upon g and n) that you know the solution to an agreed upon system of equation, without revealing the solution**</font>

In [18]:
# 1. Set up Phase
# Agree upon a large prime number n and a generator g for a cyclic group modulo n.
# These values are public and shared between the prover and verifier.
n = 1613
g = 3

result1 = 7944
result2 = 4764

# 2. Prover's Phase:
# Known solution (known only to the prover)
x = 420
y = 888

print("only reveal g^x mod n and g^y mod n ")
# g^x mod n
g_x = pow(g, x, n)
print("g^x mod n is {}".format(g_x))
# g^y mod n
g_y = pow(g, y, n)
print("g^y mod n is {}".format(g_y))


# 2. Verifier's Phase:
# # The verifier checks if homomorphism holds by looking only the sum of the encrypted numbers. :
(pow(g_x, 2, n) * pow(g_y, 8, n)) % n == pow(g, 7944, n)
(pow(g_x, 5, n) * pow(g_y, 3, n)) % n == pow(g, 4764, n)

def verifier(statement1, statement2, witnessx, witnessy):
    # x!^* = g^* (mod p)
    temp1x = pow(witnessx, 2, n)
    temp1y = pow(witnessy, 8, n)
    temp2x = pow(witnessx, 5, n)
    temp2y = pow(witnessy, 3, n)
    
    if temp1x * temp1y %n == pow(g,statement1,n) and temp2x * temp2y %n == pow(g, statement2, n):
        return True
    return False

verifier(7944, 4764, pow(g, x, n), pow(g, y, n))

only reveal g^x mod n and g^y mod n 
g^x mod n is 1030
g^y mod n is 769


True

## The field of rational numbers ℚ is homomorphic to the field ℤp

<font color='red'>**Compute 53/192 + 61/511 (mod 1033) in python. It is not required to implement euclid's algorithm.**</font>

In [23]:
n = 1033
g = 5
# Define the fractions and modulus
numerator1 = 53
denominator1 = 192
numerator2 = 61
denominator2 = 511


# Group A is homomorphic to group B if there exists a transformation φ
# where φ maps elements from A to B, and for all a, a’ in A, φ(a □ a’) = φ(a) △ φ(a’).

# The transformation φ (x) is φ(x) = numerator * denominator ** -1 % n

# ( ℚ ,+ , *)
# Group (or Ring) A is rational number ℚ  under: addition.
# ( ℤ / ℤp, ,+ , *)
# Group (or Ring) B is ℤ / ℤp  under: addition modulo p.


int1 = numerator1
Z1 = pow(denominator1,-1,n)
a_1 = int1 * Z1 % n

print("a_1: φ (a) is {}".format(a_1))

int2 = numerator2
Z2 = pow(denominator2,-1,n)
a_2 = int2 * Z2 % n

print("a_2: φ (a') is {}".format(a_2))

# print(pow(g, a, n) * pow(g, b, n) == pow(g, a + b, n))

LHS = (int1*Z1 +  int2*Z2) % 1033
print("LHS: φ (a) + φ (a') is  {}".format(LHS))

print("new num {}".format(38795))
print("new den {}".format(98112))
print("new res {}".format(38795/98112))

n_int = 38795
n_z = pow(98112,-1,n)
RHS = n_int * n_z % n
print("RHS: φ (a + a')  is {}".format(RHS))


# LHS = (numerator1/denominator1) + (numerator2/denominator2)
# RHS = ( (numerator1 % n) * (denominator1 ** (-1) % n) +  (numerator2 % n) * (denominator2 ** (-1) % n)  ) % n


# print("LHS is {}".format(LHS))
# print("RHS is {}".format(RHS))


# print("φ (a + a')  is {}".format( ( numerator1/denominator1 + numerator2/denominator2 ) % n ))
# print("φ (a) + φ (a') is {}".format( ( (numerator1 % n) * (denominator1 ** (-1) % n) +  (numerator2 % n) * (denominator2 ** (-1) % n)  ) % n ) )


(LHS == RHS)
# (( numerator1/denominator1 + numerator2/denominator2 ) % n) == ( (numerator1 % n) * pow(denominator1, -1, n) +  (numerator2 % n) * pow(denominator2, -1, n)   % n )

a_1: φ (a) is 619
a_2: φ (a') is 928
LHS: φ (a) + φ (a') is  514
new num 38795
new den 98112
new res 0.39541544357469016
RHS: φ (a + a')  is 514


True